In [29]:
from web3 import Web3, HTTPProvider

In [30]:
# load abi and bytecode
with open('../contracts/auction/abi.json', 'r') as abi_file:
    abi = abi_file.read()
with open('../contracts/auction/bytecode', 'r') as bc:
    bytecode = bc.read()

In [31]:
# web3.py instance
w3 = Web3(HTTPProvider('http://localhost:8545'))

In [32]:
# account data
owner_account_address = w3.toChecksumAddress('0xdc88f96bcbedbd92539fefe2c91148cb67d272e4')
guest_account_address = w3.toChecksumAddress('0x0635c434ec5f88f7dc8e44e0b878116d71d2a658')
owner_account_password = 'password'
guest_account_password = 'second_password'

In [33]:
# prepare contract object
auction_deploy = w3.eth.contract(abi=abi, bytecode=bytecode)

In [34]:
# get address balance before auction
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

768258168000001001 wei
0.768258168000001001 ether


In [35]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)
# Submit the transaction which deploys the contract with biddingTime 300 seconds (5 minutes)
tx_hash = auction_deploy.constructor(300).transact({
    'from': owner_account_address
})
print(tx_hash.hex())

0xebc933b096e185d5f60c80bac256e3d1f1707e0decd09e123a48b56c71fa6395


In [36]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt)

AttributeDict({'blockHash': HexBytes('0x5db333239434df7d4d18035738a8637fefc55cc53770b644cda02535fb6e8ba7'), 'blockNumber': 5682282, 'contractAddress': '0x4ab8Dda9Af5eA9D8d02bc0735066a48B11F5cFC8', 'cumulativeGasUsed': 6784121, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 452494, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': None, 'transactionHash': HexBytes('0xebc933b096e185d5f60c80bac256e3d1f1707e0decd09e123a48b56c71fa6395'), 'transactionIndex': 48}

In [37]:
# Create the contract instance with the newly-deployed address (can't use without address)
contract = w3.eth.contract(
    address = w3.toChecksumAddress(tx_receipt['contractAddress']),
    abi=abi,
)

In [38]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [39]:
# prepare contract function object
bid_obj = contract.functions.bid()
bid_gas_estimated = bid_obj.estimateGas({
    'from': owner_account_address,
    'value': Web3.toWei(0.03, 'ether')
})
print(w3.eth.gasPrice, 'current gas price in Wei, testnet')
print(bid_gas_estimated, 'estimated for the transaction')
transaction_cost = float(Web3.fromWei(w3.eth.gasPrice * bid_gas_estimated, 'ether')) * 132.45
print(transaction_cost, 'payed for the transactions in USD')

1000000000 current gas price in Wei, testnet
63651 estimated for the transaction
0.00843057495 payed for the transactions in USD
0.00843057495 payed for the transactions in USD


In [40]:
# 1 ether 132,45$ | 3.97$ == 0.03 == 30 000 000 000 000 000 wei
bid_tx = bid_obj.transact({
    'from': owner_account_address,
    'value': Web3.toWei(0.03, 'ether'),
    'gas': bid_gas_estimated,
    'gas_price': w3.eth.gasPrice
})
print(bid_tx.hex())

0xafbc3d221cbf2bfd6486adcae536411e4866db040652f8da25a5e1ac24ccb913


In [41]:
# Wait for the transaction to be mined, and get the transaction receipt
print(w3.eth.waitForTransactionReceipt(bid_tx))

AttributeDict({'blockHash': HexBytes('0xd0263cd43f9386410b97bf60b1afe23a63e4ea7923775bcd0266aac9f61652f5'), 'blockNumber': 5682293, 'contractAddress': None, 'cumulativeGasUsed': 6951541, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 63651, 'logs': [AttributeDict({'address': '0x4ab8Dda9Af5eA9D8d02bc0735066a48B11F5cFC8', 'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf0809674f6aab8ad300')], 'data': '0x000000000000000000000000dc88f96bcbedbd92539fefe2c91148cb67d272e4000000000000000000000000000000000000000000000000006a94d74f430000', 'blockNumber': 5682293, 'transactionHash': HexBytes('0xafbc3d221cbf2bfd6486adcae536411e4866db040652f8da25a5e1ac24ccb913'), 'transactionIndex': 62, 'blockHash': HexBytes('0xd0263cd43f9386410b97bf60b1afe23a63e4ea7923775bcd0266aac9f61652f5'), 'logIndex': 77, 'removed': False})], 'logsBloom': HexBytes('0x0000000000000000000000000080000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000

In [42]:
# get highestBidder using call()
print(contract.functions.highestBidder().call(), 'highestBidder')
# get highestBid
print(contract.functions.highestBid().call(), 'highestBid')

0xdC88f96BCBEdbD92539Fefe2c91148Cb67d272E4 highestBidder
30000000000000000 highestBid


In [43]:
# make another bid with other account, bid is less then previous
w3.personal.unlockAccount(guest_account_address, guest_account_password)

True

In [44]:
# 1 ether 132,45$ | 5.30$ == 0.040 == 40 000 000 000 000 000 wei
bid_tx = bid_obj.transact({
    'from': guest_account_address,
    'value': Web3.toWei(0.040, 'ether'),
})
print(w3.eth.waitForTransactionReceipt(bid_tx))

AttributeDict({'blockHash': HexBytes('0x59dd64f3858e18d6235a322a7570e572ff65b637b224e1a93a4ae5fa3b737911'), 'blockNumber': 5682298, 'contractAddress': None, 'cumulativeGasUsed': 2931540, 'from': '0x0635c434ec5f88f7dc8e44e0b878116d71d2a658', 'gasUsed': 54416, 'logs': [AttributeDict({'address': '0x4ab8Dda9Af5eA9D8d02bc0735066a48B11F5cFC8', 'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf0809674f6aab8ad300')], 'data': '0x0000000000000000000000000635c434ec5f88f7dc8e44e0b878116d71d2a658000000000000000000000000000000000000000000000000008e1bc9bf040000', 'blockNumber': 5682298, 'transactionHash': HexBytes('0x8e829a28298c2f4133b0db8f23b2210fd5bdde4bc103ffe15973a99b58a81928'), 'transactionIndex': 7, 'blockHash': HexBytes('0x59dd64f3858e18d6235a322a7570e572ff65b637b224e1a93a4ae5fa3b737911'), 'logIndex': 45, 'removed': False})], 'logsBloom': HexBytes('0x00000000000000000000000000800000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000

In [45]:
# get highestBidder using call()
print(contract.functions.highestBidder().call(), 'highestBidder')
# get highestBid
print(contract.functions.highestBid().call(), 'highestBid')

0x0635c434EC5F88F7Dc8E44E0B878116d71D2A658 highestBidder
40000000000000000 highestBid


In [46]:
# get address balance before withdraw
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

737742023000001001 wei
0.737742023000001001 ether


In [47]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [48]:
# withdraw bid, owner
withdraw_obj = contract.functions.withdraw()
withdraw_tx = withdraw_obj.transact({
    'from': owner_account_address,
})
print(withdraw_tx.hex())

0x3d111a5ebb35efefdeccdf6a9a54863a676b35fd70c5ef0681ad508be23ab82e


In [49]:
# Wait for the transaction to be mined, and get the transaction receipt
print(w3.eth.waitForTransactionReceipt(withdraw_tx))

AttributeDict({'blockHash': HexBytes('0x0f1c8b261321ca832a17c7636fc0e8b63d4907905f78d99608092ce5fcb3de9a'), 'blockNumber': 5682302, 'contractAddress': None, 'cumulativeGasUsed': 6997836, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 19410, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x4ab8dda9af5ea9d8d02bc0735066a48b11f5cfc8', 'transactionHash': HexBytes('0x3d111a5ebb35efefdeccdf6a9a54863a676b35fd70c5ef0681ad508be23ab82e'), 'transactionIndex': 42})

In [50]:
# get address balance after withdraw
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

767722613000001001 wei
0.767722613000001001 ether


In [55]:
# get auctionEndTime using call()
auction_end_time = contract.functions.auctionEndTime().call()
print(auction_end_time)
from datetime import datetime
print(datetime.utcfromtimestamp(auction_end_time).strftime('%Y-%m-%d %H:%M:%S'))

1558974963
2019-05-27 16:36:03


In [56]:
# get beneficiary address
print(contract.functions.beneficiary().call())

0xdC88f96BCBEdbD92539Fefe2c91148Cb67d272E4


In [57]:
# unlock account to transact
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [58]:
# finish auction if now >= auctionEndTime, transfer data of highestBid to auction owner
auction_end_obj = contract.functions.auctionEnd()
auction_end_tx = auction_end_obj.transact({
    'from': owner_account_address,
})
print(w3.eth.waitForTransactionReceipt(auction_end_tx.hex()))

AttributeDict({'blockHash': HexBytes('0xc7bd23934e6f495bd0cb3db16db537b2dab13d2c6886299d24582b7bcf25754b'), 'blockNumber': 5682310, 'contractAddress': None, 'cumulativeGasUsed': 5225893, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 51919, 'logs': [AttributeDict({'address': '0x4ab8Dda9Af5eA9D8d02bc0735066a48B11F5cFC8', 'topics': [HexBytes('0xdaec4582d5d9595688c8c98545fdd1c696d41c6aeaeb636737e84ed2f5c00eda')], 'data': '0x0000000000000000000000000635c434ec5f88f7dc8e44e0b878116d71d2a658000000000000000000000000000000000000000000000000008e1bc9bf040000', 'blockNumber': 5682310, 'transactionHash': HexBytes('0x63297af797d7a0e554d3e202b869d2d0812db3a79d2c78bef636f199191bad1a'), 'transactionIndex': 42, 'blockHash': HexBytes('0xc7bd23934e6f495bd0cb3db16db537b2dab13d2c6886299d24582b7bcf25754b'), 'logIndex': 29, 'removed': False})], 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000

In [59]:
# get address balance after auction
owner_balance = w3.eth.getBalance(owner_account_address)
print(Web3.fromWei(begin_owner_balance, 'ether'), 'begin balance ether')
print(Web3.fromWei(owner_balance, 'ether'), 'end balance ether')

0.767722613000001001 begin balance ether
0.807670694000001001 end balance ether
